# 1. HPC PDE Solver - The data part

## Requirements

In order to fully understand this notebook, please be sure to have already taken a look at the notebook:
* **0_hpc_mpi_tutorial**

**It is also adviced to run this Jupyter notebook, for the first time, with 4 processes as the examples are made for 4 processes** but it would work for any number of processes. 

## Fisher's equation

This is the first part of our MPI mini-app. As a quick reminder, the mini-app is a solver for the partial differential equation of Fisher. This equation represents a phenonema of reaction-diffusion. Several real-life phenomenon can be represented by it such as:
* The dynamic (growth) of a population, for example, of rabbits. 
* The diffusion of some chemical components like in a combustion for example.
* The diffusion of an advantageous gene among a certain population. It was, actually, the initial aim for the developement of the equation.

The equation has the form: 

$\frac{\partial s}{\partial t} = D(\frac{\partial^{2} s}{\partial x^{2}} + \frac{\partial^{2} s}{\partial y^{2}})+Rs(1-s)$

Where: 
* $D$ is the diffusion coefficient (rate of diffusion). $D(\frac{\partial^{2} s}{\partial x^{2}} + \frac{\partial^{2} s}{\partial y^{2}})$ represents the diffusion part of the equation.
* $R$ is the reaction coefficient (intensity of the reaction). $Rs(1-s)$ represents the reaction part of the equation.
* $s$ is the representation of the species concentration (in the case of species) between 0 and 1. A 0 means that no rabbit is in the zone and 1 means that the zone has reached the maximum number of rabbits possible.

The problem is that, in its original form, the equation can not be solved by a computer as it is a differential equation which is continuously changing. Thus, we have to transform it a bit. We will **basically** discretize the 3 variables on which the equation is differentiated, namely, $x$, $y$ and $t$. The discretization of those variables simply means that we will restrict them on some finite domain. 
* $x$ and $y$ represents the 2D space of the diffusion (evolution of the diffusion over space). We will discretize them by creating a finite dimension grid $i \times j$.
* $t$ represents the time of the diffusion (evolution of the diffusion over time), we will discretize it by defining a finite number of time steps $k$.

Consequently, $s$ will become $s_{i,j}^{k}$, namely, the $s$ value (between 0 and 1) at grid point $(i,j)$ for time step $k$. 

## The data part of the mini-app

The first fundemental part of this mini-app is the way the data structures are defined. Indeed, it is very important to choose a model (a structure) for splitting the job among the processes as it will constitutes the heart of our app. The rest being only informations transferred between the processes and computations. 

### The idea

When running the simulation, you have to choose a grid size, the grid represents the discretization of the problem. For simplicity, let's admit that you choose a square grid of **8x8 (64 nodes in total)** on which you want to run the simulation. You want, now, to make a fair division of the computations between each process. In order to do so, what you can do is dividing the grid into a number of subdomains corresponding to the number of processes you have in parallel. Let's admit that you have **4 processes** in parallel. Thus a $8 \times 8$ grid would look like this: 

![title](media/picture1.png)

Process 0 will, thus, take care of calculating the $S$ values for each of **its grid points (4x4)** and the principle is the same for the other processes, it is a fair distribution of the work. Yes, the boundary points of the domains will have to know the values of the points on the neighbouring domains in order to calculate their $S$ values, but we will come to that later. 

### The code

First, let's initialize the Ipyparallel client in order to get the ability to use Mpi4Py in Jupyter Lab. 

In [1]:
import ipyparallel as ipp
rc = ipp.Client()
print("There are " + str(len(rc)) + " processes running in parallel")
print("IDs of the processes running in parallel", rc.ids)

There are 4 processes running in parallel
IDs of the processes running in parallel [0, 1, 2, 3]


From now on, we can use the magic command **%%px** with Mpi4py in order to run code in the cell in all the processes simultaneously. Let's import **Mpi4Py** and of course **Numpy** as we will heavily use matrices. 

In [2]:
%%px
from mpi4py import MPI as mpi
import numpy as np
import time

Let's initialize MPI: 

In [3]:
%%px

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

print("Hello from process " + str(rank))

[stdout:0] Hello from process 0
[stdout:1] Hello from process 1
[stdout:2] Hello from process 2
[stdout:3] Hello from process 3


We will bascially define 2 objects to handle the data of our mini-app. Both will exist in every process executing the code:
* The **discretization object** will handle all data related to the full grid and to the simulation. **In other terms, this object will handle all parameters and variables which are the same among all processes. Each discretization object will have exactly the same attributes among all processes.**
* The **domain object** will handle the data proper to each domain of each process. **Each domain object will have unique values for their attributes in each processes as each domain is unique**. 

### The discretization object

As you know each piece of code is executed among all processes. Thus, the discretization object will exist in each process. For now, the object is just initialized and the variables "declared". Look at the code and be careful, the **x dimension is the horizontal number of grid points** and the **y dimension is the vertical number of grid points**. It is important for the matrices later. 

When creating the simulation, the minimal parameters are: 
* The number of horizontal grid points: $nx$.
* The number of vertical grid points: $ny$. 
* The number of time steps: $nt$
* The total time of the simulation: $t$

Optionally, you can pass: 
* The D coefficient: $d$
* the R coefficient: $r$
* The verbose output parameter (0 or 1) to toggle verbose mode: $v$
* A list of points for intializing your own customised initial conditions: $ic$.

In [4]:
%%px

# here we pass the mandatory parameters for creating the discretization object: a grid of 8x8 with 100 time steps and a total time of 0.01
sim = [8, 8, 100, 0.01]

We can thus build the **discretization object** and highlight some facts: 
* For $dx$, the $1.0$ comes from the fact that we assume the horizontal length of the grid to be always equal to $1$. We divide by $(nx-1)$ because if we have 5 grid points, there is 4 interpoint spaces (just draw them if you are not sure). 
* For the custom initial condition, the list of points should be a list of lists of the form: $[[x_{1},y_{1},s_{1}],[x_{2},y_{2},s_{2}], ...]$ where $x$ and $y$ are the coordinates of that point and $s$ the initial $S$ value. If the custom initial condition is not specified, the default one will be used. As you know, the default one is a circle not bigger than $\frac{1}{8}$ of the full grid size on the bottom left of this latter. 

In [5]:
%%px

class Discretization: 
    
    def __init__(self, nx, ny, nt, t, d=1.0, r=1000.0, v=0, ic=None):
        self.nx = int(nx)                             # x dimension (Number of HORIZONTAL grid points) (int)
        self.ny = int(ny)                             # y dimension (Number of VERTICAL grid points) (int)
        self.nt = int(nt)                             # number of time steps (int)
        self.dx = 1.0 / (nx - 1)                      # distance between each grid points (vertically and horizontally) (double)
        self.dt = t / nt                              # time step size (double)
        self.d = d                                    # the diffusion coefficient D (double)
        self.r = r                                    # the reaction coefficient R (double)
        self.alpha = (self.dx**2)/(self.d * self.dt)  # dx^2/(D*dt) (double)
        self.beta = self.r * (self.dx**2) / self.d    # R*dx^2/D (double)
        
        # simulation parameters
        self.verbose_output = False      # if you want to print more informations during the simulation
        self.custom_init = False         # if the simulation has a customised initial condition
        
        # verbose output
        if v != 0:
            self.verbose_output = True 
        
        # we check that the points entered are valid for the customised initial condition
        # None is equivalent to False when evaluated in Python
        if ic: 
            for point in ic:
                    assert(point[0] >= 0 and point[0] < self.nx)  # checking x coordinate
                    assert(point[1] >= 0 and point[1] < self.ny)  # checking y coordinate
                    assert(point[2] >= 0.0 and point[2] <= 1.0)   # checking s value
            self.points = ic                                        # the custom initial condition is valid
            self.custom_init = True
       
        # statistics variables initialized
        self.flops_count = 0   # number of flops
        self.iters_cg = 0      # number of Conjugate Gradient iteration
        self.iters_newton = 0  # number of Newton iteration

Let's do a test by printing the value for process 0. You can change the rank in the condition to see that all the processes have the same values for the discretization object:

In [6]:
%%px

test = Discretization(sim[0], sim[1], sim[2], sim[3])

if rank == 0:
    print("nx:", test.nx)
    print("ny:", test.ny)
    print("nt:", test.nt)    
    print("dx:", test.dx)
    print("dt:", test.dt)
    print("d:", test.d)
    print("r:", test.r)
    print("alpha:", test.alpha)
    print("beta:", test.beta)
    print("Custom initial condition:", test.custom_init)

[stdout:0] 
nx: 8
ny: 8
nt: 100
dx: 0.14285714285714285
dt: 0.0001
d: 1.0
r: 1000.0
alpha: 204.0816326530612
beta: 20.408163265306122
Custom initial condition: False


### The domain object

Now let's do the per process unique object, the domain object. This latter will handle all the informations of the proper domain of each process as shown in the illustration above. **At first glance, it may seems difficult to create a general handler which will work for any number of processes and any number of grid points, but we will use an MPI command which will make the whole logic a lot easier !**.

This function is `Create_cart()` ! It creates a cartesian topoly of the given available processes. Remember `COMM_WORLD` wich was a subset of all processes, `Create_cart()` is also a subset of all processes but it wraps them into a different structure forming a cartesian topology. Let's say we have 4 processes, it will return a 2x2 cartesian topolgy object of the processes. The MPI Python definition of the `Create_cart()` command is the following: 

`def Create_cart(self, dims, periods=None, bint reorder=False):`

* **dims**: a list of 2 integers defining the number of processes along the 2 dimensions of the topolgy. Ex: `[2, 2]` is a 2x2 cartesian plan.
* **periods**: a list of 2 boolean integers (0 or 1) defining if when reaching the last process of each dimension, the next one is the first of that dimension (it loops back) or if there is no next one. Ex: `[0, 0]` mean that there is no periodicity over the dimensions). 
* **reorder**: an integer (0 or 1) defining if we want our processes to be reordered optimally or not (we ignore this parameter in the mini-app).

The problem with this MPI command is that we have to manually input the dimensions of the topolgy. For example, for 4 processes, we have to input in the function a `[2,2]` (2x2) or a `[1, 4]` (1x4) parameter manually, the function does not do it for us... So let's create our own function for defining the dimensions of the cartesian plan given any number of available processes: 

In [7]:
%%px

class Domain: 

    # a simple function which generates the dimensions of the cartesian plan given a number of processors (size)
    def create_dim(self, size):
        
        # we list all dividers
        dividers = []
        for i in range(size - 1 , 0, -1):
            if (size%i == 0 and i != 1):
                dividers.append(i)
        # let's print the dividers to see how it works
        print("Dividers of " + str(size) + " without 1 are " + str(dividers))
        
        # if the number is prime (no divider)
        # we return the number itself and the dimension 1
        if (dividers == []):
            return [size, 1]
        # else we return the divider(s) in the middle of the list to have a balanced division
        else: 
            divider = dividers[len(dividers) // 2]
            return [size // divider, divider]

In [8]:
%%px

test = Domain()

if rank == 0:
    print("The dimensions for 1 processes are:", test.create_dim(1), "\n")
    print("The dimensions for 4 processes are:", test.create_dim(4), "\n")
    print("The dimensions for 13 processes are:", test.create_dim(13), "\n")
    print("The dimensions for 45 processes are:", test.create_dim(45), "\n")
    print("The dimensions for 60 processes are:", test.create_dim(60), "\n")
    print("The dimensions for 100 processes are:", test.create_dim(100), "\n")

[stdout:0] 
Dividers of 1 without 1 are []
The dimensions for 1 processes are: [1, 1] 

Dividers of 4 without 1 are [2]
The dimensions for 4 processes are: [2, 2] 

Dividers of 13 without 1 are []
The dimensions for 13 processes are: [13, 1] 

Dividers of 45 without 1 are [15, 9, 5, 3]
The dimensions for 45 processes are: [9, 5] 

Dividers of 60 without 1 are [30, 20, 15, 12, 10, 6, 5, 4, 3, 2]
The dimensions for 60 processes are: [10, 6] 

Dividers of 100 without 1 are [50, 25, 20, 10, 5, 4, 2]
The dimensions for 100 processes are: [10, 10] 



Now that we have our function for generating the dimensions, we can build the constructor function of the Domain object. Here are some facts that have to be highlighted:
* `x_new` and `x_old` are the matrices for, respectively, the solution for the domain at timestep $k$ and $k-1$ as we also need the solution of the previous time step to calculate the new one. 
* `bndN`, `bndS`, `bndE`, `bndW` are the **vectors** for holding the boundary points of the neighbouring domains when they are received (more details on this later). 
* `buffN`, `buffS`, `buffE`, `buffW` are **vectors** used for communicating (sending) the boundary points of the current domain to the neighbouring domains (more details on this later).

In order to visualize the parameters, we will also create a print function for the domain object.

In [9]:
%%px

class Domain: 

    def create_dim(self, size):
        dividers = []
        for i in range(size - 1 , 0, -1):
            if (size%i == 0 and i != 1):
                dividers.append(i)
        if (dividers == []):
            return [size, 1]
        else: 
            divider = dividers[len(dividers) // 2]
            return [size // divider, divider]
    
    # constructor for the domain object
    def __init__(self, rank, size, discretization, communicator):

        # dimensions of the cartesian plan
        dims = self.create_dim(size)
        
        # save the dimension on each axis as an object attribute
        self.ndomy = dims[0]
        self.ndomx = dims[1]

        # generate the cartesian plan given the dimensions (mpi function)
        # no periodicity of the dimensions
        # no reordering
        comm_cart = communicator.Create_cart(dims, [False, False], False)
        
        # save the cartesian communication group as an object attribute
        self.comm_cart = comm_cart

        # get the coordinates of the current processor on the newly generated cartesian plan (mpi function)
        coords = self.comm_cart.Get_coords(rank)
        
        # save the coordinates as attributes
        self.domy = coords[0] # (int)
        self.domx = coords[1] # (int)
        
        # get the ranks of the domains neighbouring the current domain
        # .Shift(direction_on_cartesian_plan, size_of_the_shift)
        # row direction is 0 (x-direction)
        # column direction is 1 (y-direction)
        south_north = self.comm_cart.Shift(0, 1) # tuple (int, int)
        west_east = self.comm_cart.Shift(1, 1) # tuple (int, int)
    
        # save the rank of neighbouring domains as object attributes
        self.neighbour_south = south_north[0]
        self.neighbour_north = south_north[1]
        self.neighbour_west = west_east[0]
        self.neighbour_east = west_east[1]

        # number of horizontal and vertical grid points of the current domain
        self.nx = discretization.nx // self.ndomx # (int)
        self.ny = discretization.ny // self.ndomy # (int)

        # the starting coordinates of the current (sub)domain in the full grid
        # the coordinates start from 0
        self.startx = (self.domx) * self.nx # (int)
        self.starty = (self.domy) * self.ny # (int)

        # we adjust for grid dimensions that, potentially, do not divide evenly between the domains
        # we stretch the last element of each dimension to fit the remaining part
        # domx and domy start from 0 so we have to do (ndomx - 1) & (ndomy - 1)
        if self.domx == (self.ndomx - 1):
            self.nx = discretization.nx - self.startx
        if self.domy == (self.ndomy - 1):
            self.ny = discretization.ny - self.starty
        
        # the ending coordinates in the full grid of the current domain
        self.endx = self.startx + self.nx - 1 # (int)
        self.endy = self.starty + self.ny - 1 # (int)

        # the total number of grid points of the current domain
        self.n_total = self.nx * self.ny # (int)

        # mpi values for the current domain saved as object attributes
        self.rank = rank
        self.size = size
        
        # fields holding the solutions (Numpy matrices)
        self.x_new = np.zeros((self.ny, self.nx), dtype=np.float64) # solution at timestep k (2d)
        self.x_old = np.zeros((self.ny, self.nx), dtype=np.float64) # solution at timestep k-1 (2d)
       
        # fields holding the boundary points when they are received from the neighbouring domains
        self.bndN = np.zeros((1, self.nx), dtype=np.float64) # boundary north (1d)
        self.bndS = np.zeros((1, self.nx), dtype=np.float64) # boundary east (1d)
        self.bndE = np.zeros((1, self.ny), dtype=np.float64) # boundary south (1d)
        self.bndW = np.zeros((1, self.ny), dtype=np.float64) # boundary east (1d)

        # buffers used during boundaries communication
        # to send the boundary points of the current domain
        # to its neighbours
        self.buffN = np.zeros((1, self.nx), dtype=np.float64) # (1d)
        self.buffS = np.zeros((1, self.nx), dtype=np.float64) # (1d)
        self.buffE = np.zeros((1, self.ny), dtype=np.float64) # (1d)
        self.buffW = np.zeros((1, self.ny), dtype=np.float64) # (1d)
     
    # function for printing the parameters of the cartesian division among each process
    def print(self):
        # print, for each process, the domain object
        for i in range(0, self.size):
            if i == self.rank:  
                print("Rank " + str(self.rank) + "/" + str(self.size-1))
                print("At index (" + str(self.domy) + "," + str(self.domx) + ")")
                print("Neigh N:S " + str(self.neighbour_north) + ":" + str(self.neighbour_south))
                print("Neigh E:W " + str(self.neighbour_east) + ":" + str(self.neighbour_west))
                print("Coordinates startx:endx  " + str(self.startx) + ":" + str(self.endx))
                print("Coordinates starty:endy  " + str(self.starty) + ":" + str(self.endy))
                print("Local dims " + str(self.nx) + " x " + str(self.ny))
                print("")
            
            # keep the printing ordered and cleaned
            MPI.COMM_WORLD.Barrier()
            # wait a bit when printing to avoid polluating other printed messages.
            time.sleep(0.1)

        return None

Let's test it with our domain of 8x8 and our 4 MPI process. 

In [10]:
%%px

discretization = Discretization(sim[0], sim[1], sim[2], sim[3])
domain = Domain(rank, size, discretization, comm)
domain.print()

[stdout:0] 
Rank 0/3
At index (0,0)
Neigh N:S 2:-1
Neigh E:W 1:-1
Coordinates startx:endx  0:3
Coordinates starty:endy  0:3
Local dims 4 x 4

[stdout:1] 
Rank 1/3
At index (0,1)
Neigh N:S 3:-1
Neigh E:W -1:0
Coordinates startx:endx  4:7
Coordinates starty:endy  0:3
Local dims 4 x 4

[stdout:2] 
Rank 2/3
At index (1,0)
Neigh N:S -1:0
Neigh E:W 3:-1
Coordinates startx:endx  0:3
Coordinates starty:endy  4:7
Local dims 4 x 4

[stdout:3] 
Rank 3/3
At index (1,1)
Neigh N:S -1:1
Neigh E:W -1:2
Coordinates startx:endx  4:7
Coordinates starty:endy  4:7
Local dims 4 x 4



For 4 processes, the cartesian topology should be as follows:

![title](media/picture2.png)

Let's try with some other parameters to see how the grid is divided:

In [11]:
%%px

sim = [128, 128, 100, 0.01]
discretization = Discretization(sim[0], sim[1], sim[2], sim[3])
domain = Domain(rank, size, discretization, comm)
domain.print()

[stdout:0] 
Rank 0/3
At index (0,0)
Neigh N:S 2:-1
Neigh E:W 1:-1
Coordinates startx:endx  0:63
Coordinates starty:endy  0:63
Local dims 64 x 64

[stdout:1] 
Rank 1/3
At index (0,1)
Neigh N:S 3:-1
Neigh E:W -1:0
Coordinates startx:endx  64:127
Coordinates starty:endy  0:63
Local dims 64 x 64

[stdout:2] 
Rank 2/3
At index (1,0)
Neigh N:S -1:0
Neigh E:W 3:-1
Coordinates startx:endx  0:63
Coordinates starty:endy  64:127
Local dims 64 x 64

[stdout:3] 
Rank 3/3
At index (1,1)
Neigh N:S -1:1
Neigh E:W -1:2
Coordinates startx:endx  64:127
Coordinates starty:endy  64:127
Local dims 64 x 64



You should have now a good view of what is going on with the data structures of our mini-app ! Let's move on and run into the diffusion part !